In [1]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import pandas as pd

customer_path = 'dataset/customer.csv'
sales_path = 'dataset/sales.csv'
goods_path = 'dataset/goods.csv'
category_path = 'dataset/x_tree.csv'
publisher_path = 'dataset/x_tag.csv'

customer_df = pd.read_csv(customer_path)
sales_df = pd.read_csv(sales_path)
goods_df = pd.read_csv(goods_path)
category_df = pd.read_csv(category_path)
publisher_df = pd.read_csv(publisher_path)

### Create Goods Feature

In [2]:
goods_df.head(10)

,good_id,brand,author,translator,catgory,tag
0,162,96,115,NaN,153,1|1884|1965|3728|48864
1,163,96,115,NaN,66,1|434|438|441|454|518|552|727|1355|1480|1884|1...
2,165,96,1724,NaN,139|141,1|1884|1965|3728|48864
3,166,96,2043,NaN,118,1|1884|1965|3728|48864
4,167,96,1829,NaN,139|140,1|1884|1965|3728|48864
5,168,96,1724,NaN,130,1|1884|1965|3728|48864
6,169,96,1847,NaN,120,1|1884|1965|3728|48864
7,170,96,1647,13744,167,1|1884|1965|3728|48864
8,171,96,115,NaN,64|73,1|1884|1965|3728|48864
9,172,96,115,NaN,62|65|66,1|1884|1965|3728|48864|56067


In [5]:
category = goods_df.catgory.str.get_dummies("|")
category

,100,101,102,103,104,105,106,107,108,109,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

category1 = goods_df.catgory.str.split("|").apply(lambda x: list(map(int, x)) if isinstance(x, list) else []).tolist()
mlb = MultiLabelBinarizer()
catgory_encoded = pd.DataFrame(mlb.fit_transform(category1), columns=mlb.classes_, index=goods_df.index)
catgory_encoded

,29,31,32,35,37,38,40,41,42,43,...,170,171,173,178,189,190,191,192,193,197
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57088,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57089,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57090,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57091,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
catgory_encoded.values[29]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [25]:
category.values[29]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=int64)